In [1]:
from openai import OpenAI
from redditqa.data import huggingface_dataset
from redditqa.data import qa_generation

import importlib
importlib.reload(qa_generation)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'redditqa.data.qa_generation' from '/workspaces/reddit_qa/redditqa/data/qa_generation.py'>

In [2]:
dataset = huggingface_dataset.load_reddit_dataset('/scratch1/redditqa/data/ask_historians/AskHistorians.jsonl')
dataset = qa_generation.apply(dataset)
dataset = dataset.shuffle(42)
dataset

Saving the dataset (2/2 shards): 100%|██████████| 116452/116452 [00:03<00:00, 36411.68 examples/s]
/usr/local/lib/python3.10/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Dataset({
    features: ['question_created_utc', 'question_retrieved_on', 'question_deleted', 'question_title', 'question_selftext', 'question_score', 'question_char_length', 'question_selftext_char_length', 'full_text', 'prompt', 'question', 'answer'],
    num_rows: 116452
})

In [4]:
client = OpenAI()

In [5]:
GPT_MODEL = "gpt-3.5-turbo"

GPT_PROMPT_TEMPLATE = """You are an expert historian. You are answering a question from a curious redditor. The question is:

Question: %question

Give an answer to the question."""

N_ANSWERS = 20


def prompt_openai(question):
    prompt = GPT_PROMPT_TEMPLATE.replace("%question", question)

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=GPT_MODEL,
    )

    return response.choices[0].message.content


dataset = dataset.select(range(N_ANSWERS))

for row in dataset:
    question = row["question"] + " - " + row["question_selftext"]

    gpt_answer = prompt_openai(question)

    print("Q:", question)
    print("-" * 80)
    print("A (gpt):   ", gpt_answer)
    print("-" * 80)
    print("A (reddit):", row["answer"])
    print("-" * 80)
    print("\n" * 4)

Q: What was the medical field like during the Islamic age? - 
--------------------------------------------------------------------------------
A (gpt):    During the Islamic Golden Age (8th to 14th centuries), the medical field witnessed significant advancements and contributions that impacted the course of medical knowledge throughout the world. Islamic civilization valued knowledge and scholarship, leading to the translation and preservation of Greek, Persian, and Indian medical texts. This cultural exchange fostered a rich and diverse medical tradition that built upon previous knowledge while adding new insights and discoveries.

One of the most notable advancements during this period was the establishment of hospitals, known as "bimaristans" or "maristans." These institutions provided medical care to patients regardless of their social or economic status, marking an important development in the history of healthcare. Bimaristans were often associated with mosques or educational ins